In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('prog_books.csv')

In [16]:
df1 = pd.read_csv('prog_books.csv')

In [3]:
df

,Rating,Reviews,Book_title,Description,Number_Of_Pages,Type,Price
0,4.17,"3,829",The Elements of Style,This style manual offers practical advice on i...,105,Hardcover,9.323529
1,4.01,"1,406","The Information: A History, a Theory, a Flood","James Gleick, the author of the best sellers C...",527,Hardcover,11.000000
2,3.33,0,Responsive Web Design Overview For Beginners,In Responsive Web Design Overview For Beginner...,50,Kindle Edition,11.267647
3,3.97,"1,658",Ghost in the Wires: My Adventures as the World...,If they were a hall of fame or shame for compu...,393,Hardcover,12.873529
4,4.06,"1,325",How Google Works,Both Eric Schmidt and Jonathan Rosenberg came ...,305,Kindle Edition,13.164706
...,...,...,...,...,...,...,...
284,4.13,320,Mathematics,Mathematics in designing tools coding programm...,320,Paperback,130.650000
285,4.78,365,ml,Concepts of machine learning deep learing adva...,120,Hardcover,145.900000
286,4.98,4500,dsa,data structure and algorithms analysis program...,350,Hardcover,150.360000
287,3.50,1520,matlab,MATLAB is a programming platform designed spec...,300,Paperback,96.870000


In [5]:
df1['Reviews'] = df1.Reviews.apply(lambda x: x.replace(',', '') if ',' in x else x)
df1['Reviews'] = pd.to_numeric(df1.Reviews)

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Set of stopwords to remove
stop = set(stop)

# Set of punctuation signs to remove
from string import punctuation

[nltk_data] Downloading package stopwords to C:\Users\SHBHAM
[nltk_data]     TIWARI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import re

def lower(text):
    return text.lower()

def remove_punctuation(text):
    return text.translate(str.maketrans('','', punctuation))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop])

# Removing all words with digits and standalone digits
def remove_digits(text):
    return re.sub(r'\d+', '', text)

# One function to clean it all
def clean_text(text):
    text = lower(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = remove_digits(text)
    return text

In [8]:
df['clean_Book_title']=df['Book_title'].apply(clean_text)
df.head()

,Rating,Reviews,Book_title,Description,Number_Of_Pages,Type,Price,clean_Book_title
0,4.17,"3,829",The Elements of Style,This style manual offers practical advice on i...,105,Hardcover,9.323529,elements style
1,4.01,"1,406","The Information: A History, a Theory, a Flood","James Gleick, the author of the best sellers C...",527,Hardcover,11.000000,information history theory flood
2,3.33,0,Responsive Web Design Overview For Beginners,In Responsive Web Design Overview For Beginner...,50,Kindle Edition,11.267647,responsive web design overview beginners
3,3.97,"1,658",Ghost in the Wires: My Adventures as the World...,If they were a hall of fame or shame for compu...,393,Hardcover,12.873529,ghost wires adventures worlds wanted hacker
4,4.06,"1,325",How Google Works,Both Eric Schmidt and Jonathan Rosenberg came ...,305,Kindle Edition,13.164706,google works


In [9]:
df['clean_Description']=df['Description'].apply(clean_text)
df.head()

,Rating,Reviews,Book_title,Description,Number_Of_Pages,Type,Price,clean_Book_title,clean_Description
0,4.17,"3,829",The Elements of Style,This style manual offers practical advice on i...,105,Hardcover,9.323529,elements style,style manual offers practical advice improving...
1,4.01,"1,406","The Information: A History, a Theory, a Flood","James Gleick, the author of the best sellers C...",527,Hardcover,11.000000,information history theory flood,james gleick author best sellers chaos genius ...
2,3.33,0,Responsive Web Design Overview For Beginners,In Responsive Web Design Overview For Beginner...,50,Kindle Edition,11.267647,responsive web design overview beginners,responsive web design overview beginners youll...
3,3.97,"1,658",Ghost in the Wires: My Adventures as the World...,If they were a hall of fame or shame for compu...,393,Hardcover,12.873529,ghost wires adventures worlds wanted hacker,hall fame shame computer hackers kevin mitnick...
4,4.06,"1,325",How Google Works,Both Eric Schmidt and Jonathan Rosenberg came ...,305,Kindle Edition,13.164706,google works,eric schmidt jonathan rosenberg came google se...


In [10]:
# Initializing vectorizer
vectorizer = TfidfVectorizer(analyzer='word', lowercase=False)

# Applying vectorized to clean text
X = vectorizer.fit_transform(df['clean_Book_title'])

# Getting array with vectorized titles
title_vectors = X.toarray()
title_vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
a=title_vectors

In [12]:
desc_vectorizer = TfidfVectorizer(analyzer='word', lowercase=False)
Y = desc_vectorizer.fit_transform(df['clean_Description'])
desc_vectors = Y.toarray()
desc_vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
b=desc_vectors

In [17]:
df1.columns = [i.lower() for i in df1.columns]

In [18]:
df1['reviews'] = df1.reviews.apply(lambda x: x.replace(',', '') if ',' in x else x)
df1['reviews'] = pd.to_numeric(df1.reviews)

In [19]:
dums = pd.get_dummies(df1['type'])

In [20]:
df1 = df1.merge(dums, left_index= True, right_index= True)
df1.drop('type', axis= 1, inplace= True)

In [21]:
m = df1.drop(['book_title', 'description', 'price'], axis= 1)
n = np.log2(df1.price)

In [22]:
m_train, m_test, n_train, n_test = train_test_split(m, n, test_size= .2, random_state= 42)

In [23]:
rfm = RandomForestRegressor(n_estimators = 50, random_state = 42)
rfm.fit(m_train, n_train)
n_pred= rfm.predict(m_test)

In [24]:
errors = abs(n_pred - n_test)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / n_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 0.59
Accuracy: 88.97 %.


In [25]:
scores = cross_val_score(rfm, m, n, cv=5, scoring='neg_mean_squared_error')
print(f'Cross validated random forest MAE: {abs(round(np.mean(scores), 2))}')

Cross validated random forest MAE: 1.1


In [26]:
a_train, a_test, b_train, b_test = train_test_split(a, b, test_size= .2, random_state= 42)

In [27]:
rfm = RandomForestRegressor(b_estimators = 50, random_state = 42)
rfm.fit(a_train, b_train)
b_pred= rfm.predict(a_test)

TypeError: __init__() got an unexpected keyword argument 'b_estimators'

In [57]:
def get_recommendations(value_of_element, feature_locate, df, vectors_array, feature_show):
    # Locating target element by its specific value
    index_of_element = df[df[feature_locate]==value_of_element].index.values[0]
    
    # Finding its value to show
    show_value_of_element = df.iloc[index_of_element][feature_show]

    # Dropping target element from df
    df_without = df.drop(index_of_element).reset_index().drop(['index'], axis=1)

    # Dropping target element from vectors array
    vectors_array = list(vectors_array)
    target = vectors_array.pop(index_of_element).reshape(1,-1)
    vectors_array = np.array(vectors_array)

    # Finding cosine similarity between vectors
    most_similar_sklearn = cosine_similarity(target, vectors_array)[0]

    # Sorting coefs in desc order 
    idx = (-most_similar_sklearn).argsort()

    # Finding features of similar objects by index
    all_values = df_without[[feature_show]]
    for index in idx:
      simular = all_values.values[idx]
     
    recommendations_df = pd.DataFrame({feature_show: show_value_of_element,
                                    "rec_1": simular[0][0],
                                    "rec_2": simular[1][0],
                                    "rec_3": simular[2][0],
                                    "rec_4": simular[3][0],
                                    "rec_5": simular[4][0]}, index=[0])
    

    return recommendations_df

In [58]:
get_recommendations("Algorithms", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,Algorithms,Introduction to Algorithms,Algorithms Unlocked,Data Structures and Algorithms,Computational Geometry: Algorithms and Applica...,"The Art of Computer Programming, Volume 1: Fun..."


In [30]:
get_recommendations("JavaScript", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,JavaScript,Effective JavaScript: 68 Specific Ways to Harn...,The Principles of Object-Oriented JavaScript,JavaScript: The Good Parts,Eloquent JavaScript: A Modern Introduction to ...,Sams Teach Yourself JavaScript in 24 Hours


In [31]:
get_recommendations("java", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,java,Beginning Java 2,Reviewing Java,Learn Java the Easy Way : A Hands-On Introduct...,Build Web Applications with Java: Learn every ...,Hacker's Delight


In [32]:
get_recommendations("Python", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,Python,Learning Python,Python Programming Books Set: Python Programmi...,Python Programming For Beginners: Quick And Ea...,Effective Python: 59 Specific Ways to Write Be...,Python for Data Analysis


In [33]:
get_recommendations("sql", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,sql,Practical SQL: A Beginner's Guide to Storytell...,The Elements of Style,"Fluent Python: Clear, Concise, and Effective P...",Augmented Reality: Principles and Practice,Think Like a Programmer: An Introduction to Cr...


In [34]:
get_recommendations("mysql", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,mysql,Sams Teach Yourself MySQL in 24 Hours,PHP and MySQL Web Development (Developer's Lib...,Beginners Guide On PHP Programming: Quick And ...,The Elements of Style,"Fluent Python: Clear, Concise, and Effective P..."


In [35]:
get_recommendations("C++", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,C++,The Elements of Style,Hacker's Delight,"Fluent Python: Clear, Concise, and Effective P...",Augmented Reality: Principles and Practice,Think Like a Programmer: An Introduction to Cr...


In [36]:
get_recommendations("Webdesign", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,Webdesign,The Elements of Style,Hacker's Delight,"Fluent Python: Clear, Concise, and Effective P...",Augmented Reality: Principles and Practice,Think Like a Programmer: An Introduction to Cr...


In [38]:
get_recommendations("C#", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,C#,The Elements of Style,Hacker's Delight,"Fluent Python: Clear, Concise, and Effective P...",Augmented Reality: Principles and Practice,Think Like a Programmer: An Introduction to Cr...


In [39]:
get_recommendations("toc", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,toc,The Elements of Style,Hacker's Delight,"Fluent Python: Clear, Concise, and Effective P...",Augmented Reality: Principles and Practice,Think Like a Programmer: An Introduction to Cr...


In [40]:
get_recommendations("Unix", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,Unix,The UNIX Programming Environment,Advanced Programming in the UNIX Environment,The Linux Programming Interface: A Linux and U...,The Elements of Style,"Fluent Python: Clear, Concise, and Effective P..."


In [41]:
get_recommendations("linux", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,linux,The Linux Programming Interface: A Linux and U...,The Elements of Style,"Fluent Python: Clear, Concise, and Effective P...",Augmented Reality: Principles and Practice,Think Like a Programmer: An Introduction to Cr...


In [42]:
get_recommendations("game", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,game,Cross-Platform Game Programming (Game Developm...,Game Programming Patterns,Game Programming Gems 5,Game Programming Gems 2,Game Programming Gems


In [43]:
get_recommendations("Graphics", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,Graphics,Mathematics for 3D Game Programming and Comput...,Geometric Tools for Computer Graphics,3D Math Primer for Graphics and Game Development,3D Game Engine Design: A Practical Approach to...,Augmented Reality: Principles and Practice


In [44]:
get_recommendations("Mathematics", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,Mathematics,From Mathematics to Generic Programming,Mathematics for 3D Game Programming and Comput...,Concrete Mathematics: A Foundation for Compute...,Essential Mathematics for Games and Interactiv...,Does God Play Dice?: The New Mathematics of Chaos


In [45]:
get_recommendations("ml", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,ml,The Elements of Style,Hacker's Delight,"Fluent Python: Clear, Concise, and Effective P...",Augmented Reality: Principles and Practice,Think Like a Programmer: An Introduction to Cr...


In [46]:
get_recommendations("matlab", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,matlab,Simulation of Digital Communication Systems us...,The Elements of Style,"Fluent Python: Clear, Concise, and Effective P...",Augmented Reality: Principles and Practice,Think Like a Programmer: An Introduction to Cr...


In [47]:
get_recommendations("AI", 'Book_title', df, title_vectors, 'Book_title')

,Book_title,rec_1,rec_2,rec_3,rec_4,rec_5
0,AI,The Elements of Style,Hacker's Delight,"Fluent Python: Clear, Concise, and Effective P...",Augmented Reality: Principles and Practice,Think Like a Programmer: An Introduction to Cr...


In [48]:
import pickle

In [49]:
pickle.dump(df,open('all_books.pkl','wb'))

In [50]:
df.head()

,Rating,Reviews,Book_title,Description,Number_Of_Pages,Type,Price,clean_Book_title,clean_Description
0,4.17,"3,829",The Elements of Style,This style manual offers practical advice on i...,105,Hardcover,9.323529,elements style,style manual offers practical advice improving...
1,4.01,"1,406","The Information: A History, a Theory, a Flood","James Gleick, the author of the best sellers C...",527,Hardcover,11.000000,information history theory flood,james gleick author best sellers chaos genius ...
2,3.33,0,Responsive Web Design Overview For Beginners,In Responsive Web Design Overview For Beginner...,50,Kindle Edition,11.267647,responsive web design overview beginners,responsive web design overview beginners youll...
3,3.97,"1,658",Ghost in the Wires: My Adventures as the World...,If they were a hall of fame or shame for compu...,393,Hardcover,12.873529,ghost wires adventures worlds wanted hacker,hall fame shame computer hackers kevin mitnick...
4,4.06,"1,325",How Google Works,Both Eric Schmidt and Jonathan Rosenberg came ...,305,Kindle Edition,13.164706,google works,eric schmidt jonathan rosenberg came google se...


In [52]:
title_vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [53]:
pickle.dump(a,open('a.pkl','wb'))